In [ ]:
from instagrapi import Client
from collections import namedtuple
USERNAME=""
PASSWORD=""
cl = Client()
cl.login(USERNAME, PASSWORD)
cl.account_info().dict()

In [ ]:
collections = cl.collections()
collections

**beware**, last_media_pk is misleading. collection_medias fetches all medias in the collection. Even the amount filter is only applied afterwards, so it won't optimize the query. last_media_pk is also a post-fetch filter, and it will cut the list the moment it encounters that pk. This means any non valid pk like 0 or "" will have no effect whatsoever

In [ ]:
a_col = cl.collection_medias(collection_pk="18298255798131063", amount=95, last_media_pk=0)

In [ ]:
c_col = cl.collection_medias(collection_pk="18021223057218323", amount=8161, last_media_pk="")

In [ ]:
import json, time, pydantic_core
from datetime import date

def serialize_collection(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, date):
        serial = obj.isoformat()
        return serial

    if isinstance(obj, pydantic_core._pydantic_core.Url):
        return str(obj)

with open('c_data.json', 'w') as f:
    json.dump([m.__dict__ for m in c_col], f, default=serialize_collection)